In [ ]:
%pip install ipython-sql
%load_ext sql

In [3]:
import sqlite3
import pandas as pd

Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! 

Step#1  Determine the number of member and guest bookings fr each facilityand put it into a data frame.

In [18]:
conn = sqlite3.connect(r"C:\Users\Ujval\OneDrive\Documents\GitHub\Springboard\SQLFiles Tier 2\sqlite_db_pythonsqlite.db")
cur = conn.cursor()
cur.execute("""SELECT facid, COUNT(bookid) AS count, SUM(slots) AS total,
    CASE
        WHEN memid = 0 THEN 'Guest'
        WHEN memid <> 0 THEN 'Member'
        END AS patron
   FROM Bookings
   GROUP BY facid, patron;""")
rows = cur.fetchall()
conn.close()
df = pd.DataFrame(rows)
df.columns = ['facility_id', 'booking_count', 'total_slots', 'patron_type']
print(df)

    facility_id  booking_count  total_slots patron_type
0             0            100          363       Guest
1             0            308          957      Member
2             1            113          396       Guest
3             1            276          882      Member
4             2             39          123       Guest
5             2            344         1086      Member
6             3             18           36       Guest
7             3            385          794      Member
8             4            208          520       Guest
9             4            421          884      Member
10            5             84          174       Guest
11            5             27           54      Member
12            6            245          686       Guest
13            6            195          418      Member
14            7             23           48       Guest
15            7            421          860      Member
16            8             53           54     

Step#2 Collect the prices for the different facilities in another data frame

In [22]:
conn = sqlite3.connect(r"C:\Users\Ujval\OneDrive\Documents\GitHub\Springboard\SQLFiles Tier 2\sqlite_db_pythonsqlite.db")
cur = conn.cursor()
cur.execute("""SELECT facid, name, membercost, guestcost
FROM Facilities;""")
rows = cur.fetchall()
conn.close()
prices = pd.DataFrame(rows)
prices.columns = ['facility_id', 'facility_name', 'Member', 'Guest']
print(prices)

   facility_id    facility_name  Member  Guest
0            0   Tennis Court 1     5.0   25.0
1            1   Tennis Court 2     5.0   25.0
2            2  Badminton Court     0.0   15.5
3            3     Table Tennis     0.0    5.0
4            4   Massage Room 1     9.9   80.0
5            5   Massage Room 2     9.9   80.0
6            6     Squash Court     3.5   17.5
7            7    Snooker Table     0.0    5.0
8            8       Pool Table     0.0    5.0


Step#3 Add revenue column to df

In [19]:
lookup = prices.set_index("facility_id").to_dict()
df['cost'] = df.apply(lambda x: lookup[x['patron_type']][x['facility_id']], axis=1)
df['revenue'] = df['cost'] *df['booking_count']*df['total_slots']
print(df)

    facility_id  booking_count  total_slots patron_type  cost    revenue
0             0            100          363       Guest  25.0   907500.0
1             0            308          957      Member   5.0  1473780.0
2             1            113          396       Guest  25.0  1118700.0
3             1            276          882      Member   5.0  1217160.0
4             2             39          123       Guest  15.5    74353.5
5             2            344         1086      Member   0.0        0.0
6             3             18           36       Guest   5.0     3240.0
7             3            385          794      Member   0.0        0.0
8             4            208          520       Guest  80.0  8652800.0
9             4            421          884      Member   9.9  3684423.6
10            5             84          174       Guest  80.0  1169280.0
11            5             27           54      Member   9.9    14434.2
12            6            245          686       G

Step#4 Group by facility_id and sort by revenue and remove extra rows and columns.

In [56]:
df_group = df.groupby('facility_id')['revenue'].sum()
prices['revenue'] = df_group
prices.sort_values('revenue', ascending=False, inplace=True)
prices_short = prices[['facility_name', 'revenue']]
prices_short = prices_short[prices_short['revenue'] > 1000]
prices_short.index = range(1, len(prices_short) + 1)
print(prices_short)


     facility_name     revenue
1   Massage Room 1  12337223.6
2     Squash Court   3226510.0
3   Tennis Court 1   2381280.0
4   Tennis Court 2   2335860.0
5   Massage Room 2   1183714.2
6  Badminton Court     74353.5
7       Pool Table     14310.0
8    Snooker Table      5520.0
9     Table Tennis      3240.0


The final result for Q10 gives a data frame with the facility name and corresponding revenue, sorted by descending revenue.

Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order.

Step#1: As the question does not specify, I will order this by the surname, firstname of the member, not the recommender.  I will also put the name of the recommender as one column (firstname surname).  The first step is to query the relevant information from Members.

In [106]:
conn = sqlite3.connect(r"C:\Users\Ujval\OneDrive\Documents\GitHub\Springboard\SQLFiles Tier 2\sqlite_db_pythonsqlite.db")
cur = conn.cursor()
cur.execute("""SELECT memid, surname, firstname, recommendedby
   FROM Members
   Order BY surname, firstname;""")
rows = cur.fetchall()
conn.close()
names = pd.DataFrame(rows)
names.columns = ['memid', 'surname', 'firstname', 'rec_by']
print(names)

    memid            surname  firstname rec_by
0      15              Bader   Florence      9
1      12              Baker       Anne      9
2      16              Baker    Timothy     13
3       8             Boothe        Tim      3
4       5            Butters     Gerald      1
5      22             Coplin       Joan     16
6      36            Crumpet      Erica      2
7       7               Dare      Nancy      4
8      28            Farrell      David       
9      13            Farrell     Jemima       
10      0              GUEST      GUEST       
11     20            Genting    Matthew      5
12     35               Hunt       John     30
13     11              Jones      David      4
14     26              Jones    Douglas     11
15      4           Joplette     Janice      1
16     21          Mackenzie       Anna      1
17     10               Owen    Charles      1
18     17             Pinker      David     13
19     30            Purview  Millicent      2
20      3    

Setp#2: Make a new column with the appropriate formatting for each member's name.

In [111]:
names['full_name'] = names['firstname'].str.cat(names['surname'], sep=' ')
names['full_name'] = names['full_name'] + ' (' + names['memid'].astype(str) + ')'
print(names.head())

   memid  surname firstname rec_by            full_name
0     15    Bader  Florence      9  Florence Bader (15)
1     12    Baker      Anne      9      Anne Baker (12)
2     16    Baker   Timothy     13   Timothy Baker (16)
3      8   Boothe       Tim      3       Tim Boothe (8)
4      5  Butters    Gerald      1   Gerald Butters (5)


Step#3: Match the full name to the memid of the recommender.

In [112]:
names['rec_by'] = pd.to_numeric(names['rec_by'], errors='coerce')
id_to_name = names.set_index('memid')['full_name'].to_dict()
names['recommended_by'] = names['rec_by'].map(id_to_name)
print(names.head())

   memid  surname firstname  rec_by            full_name       recommended_by
0     15    Bader  Florence     9.0  Florence Bader (15)  Ponder Stibbons (9)
1     12    Baker      Anne     9.0      Anne Baker (12)  Ponder Stibbons (9)
2     16    Baker   Timothy    13.0   Timothy Baker (16)  Jemima Farrell (13)
3      8   Boothe       Tim     3.0       Tim Boothe (8)       Tim Rownam (3)
4      5  Butters    Gerald     1.0   Gerald Butters (5)     Darren Smith (1)


Step#4: Clean up the unneccesary columns.

In [114]:
names_clean = names.drop(columns=['rec_by', 'full_name'])
names_clean['recommended_by'] = names_clean['recommended_by'].fillna('')
print(names_clean)

    memid            surname  firstname          recommended_by
0      15              Bader   Florence     Ponder Stibbons (9)
1      12              Baker       Anne     Ponder Stibbons (9)
2      16              Baker    Timothy     Jemima Farrell (13)
3       8             Boothe        Tim          Tim Rownam (3)
4       5            Butters     Gerald        Darren Smith (1)
5      22             Coplin       Joan      Timothy Baker (16)
6      36            Crumpet      Erica         Tracy Smith (2)
7       7               Dare      Nancy     Janice Joplette (4)
8      28            Farrell      David                        
9      13            Farrell     Jemima                        
10      0              GUEST      GUEST                        
11     20            Genting    Matthew      Gerald Butters (5)
12     35               Hunt       John  Millicent Purview (30)
13     11              Jones      David     Janice Joplette (4)
14     26              Jones    Douglas 

The final solution to Q11 is a report ordered by the surname, firstname of all members, with their member ID and the name of the person who recommeded them.

Q12: Find the facilities with their usage by member, but not guests.

I am interpreting this as "Find the total number of slots booked by members for each facility within the time period covered in this database".  This can be found using the same queries used above in Q10 which is stored in variables df and prices.

In [65]:
df = df[df['patron_type'] == 'Member']
df['member_usage'] = df['booking_count']*df['total_slots']
df_join = df.merge(prices, on=['facility_id'])
df_short = df_join[['facility_name', 'member_usage']].copy()
print(df_short)

     facility_name  member_usage
0   Massage Room 1        372164
1   Tennis Court 1        294756
2   Tennis Court 2        243432
3     Squash Court         81510
4   Massage Room 2          1458
5     Table Tennis        305690
6  Badminton Court        373584
7    Snooker Table        362060
8       Pool Table        670248


If desired, we can sort by member usage and reorder the index so it stays chonological.

In [66]:
df_short.sort_values('member_usage', ascending=False, inplace=True)
df_short.index = range(1, len(df_short) + 1)
print(df_short)

     facility_name  member_usage
1       Pool Table        670248
2  Badminton Court        373584
3   Massage Room 1        372164
4    Snooker Table        362060
5     Table Tennis        305690
6   Tennis Court 1        294756
7   Tennis Court 2        243432
8     Squash Court         81510
9   Massage Room 2          1458


The final result for Q11 gives a data frame with the facility name and corresponding usage by members (but not guests), sorted by descending usage.

Q13: Find the facilities usage by month, but not guests.

Step#1: I am interpreting this as "Find the total number of slots booked by members for each facility for each month within the time period covered in this database".  This first step is to query the relevant information from the data base.

In [136]:
conn = sqlite3.connect(r"C:\Users\Ujval\OneDrive\Documents\GitHub\Springboard\SQLFiles Tier 2\sqlite_db_pythonsqlite.db")
cur = conn.cursor()
cur.execute("""SELECT facid, strftime('%m', starttime) AS month, COUNT(slots) AS total_slots,
    CASE
        WHEN memid = 0 THEN 'Guest'
        WHEN memid <> 0 THEN 'Member'
        END AS patron
   FROM Bookings
   GROUP BY facid, patron, month;""")
rows = cur.fetchall()
conn.close()
months = pd.DataFrame(rows)
months.columns = ['facility_id', 'month', 'total_slots', 'patron_type']
print(months)

    facility_id month  total_slots patron_type
0             0    07           23       Guest
1             0    08           35       Guest
2             0    09           42       Guest
3             0    07           65      Member
4             0    08          111      Member
5             0    09          132      Member
6             1    07           27       Guest
7             1    08           40       Guest
8             1    09           46       Guest
9             1    07           41      Member
10            1    08          109      Member
11            1    09          126      Member
12            2    07            5       Guest
13            2    08           14       Guest
14            2    09           20       Guest
15            2    07           51      Member
16            2    08          132      Member
17            2    09          161      Member
18            3    07            3       Guest
19            3    08            4       Guest
20           

Step#2 Create new columns with the facility and month names.

In [ ]:
facid_to_name = prices.set_index('facility_id')['facility_name'].to_dict()
months['facility_name'] = months['facility_id'].map(facid_to_name)
month_dict = {1: 'Juanuary', 2:'February', 3: 'March', 4:'April', 5:'May', 6:'June', 7:'July', 8:'August', 9:'September', 10:'October', 11:'November', 12:'December'}
months['month'] = pd.to_numeric(months['month'], errors='coerce')
months['month_name'] = months['month'].map(month_dict)
print(months.head())

   facility_id  month  total_slots patron_type   facility_name month_name
0            0      7           23       Guest  Tennis Court 1       July
1            0      8           35       Guest  Tennis Court 1     August
2            0      9           42       Guest  Tennis Court 1  September
3            0      7           65      Member  Tennis Court 1       July
4            0      8          111      Member  Tennis Court 1     August


Step#3: Remove all rows for Guest booksings and order the table by facilty name (1rst) and month number (2nd). Finally, remove unnecessary columns.

In [ ]:
months_member = months[months['patron_type'] == 'Member']
months_member.sort_values(['facility_name', 'month'], ascending=True, inplace=True)
months_member = months_member[['facility_name', 'month_name', 'total_slots']]
print(months_member)

      facility_name month_name  total_slots
15  Badminton Court       July           51
16  Badminton Court     August          132
17  Badminton Court  September          161
27   Massage Room 1       July           77
28   Massage Room 1     August          153
29   Massage Room 1  September          191
33   Massage Room 2       July            4
34   Massage Room 2     August            9
35   Massage Room 2  September           14
51       Pool Table       July          103
52       Pool Table     August          272
53       Pool Table  September          408
45    Snooker Table       July           68
46    Snooker Table     August          154
47    Snooker Table  September          199
39     Squash Court       July           23
40     Squash Court     August           85
41     Squash Court  September           87
21     Table Tennis       July           48
22     Table Tennis     August          143
23     Table Tennis  September          194
3    Tennis Court 1       July  

C:\Users\Ujval\AppData\Local\Temp\ipykernel_1496\3390707986.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  months_member.sort_values(['facility_name', 'month'], ascending=True, inplace=True)


The final solution to Q13 is a table with the facility name, month and total slots booked by members.